In [1]:
import pandas as pd
import numpy as np
import json
import yaml
from statsmodels.stats.contingency_tables import mcnemar
from statsmodels.stats.contingency_tables import cochrans_q
from scipy.stats import combine_pvalues
from scipy.stats import norm

In [2]:
def read_yaml_answers(file_path):
    answers = []
    with open(file_path, "r") as f:
        documents = list(yaml.safe_load_all(f))  # handles both single list or multiple docs
        for doc in documents:
            if isinstance(doc, list):  # case: YAML file is a list of dicts
                for item in doc:
                    if isinstance(item, dict) and "answer" in item:
                        answers.append(item["answer"])
            elif isinstance(doc, dict):  # case: multiple `---` documents
                if "answer" in doc:
                    answers.append(doc["answer"])
    return answers

def read_unstructured_parsed(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    answers = []
    if isinstance(data, list):
        for obj in data:
            if isinstance(obj, dict) and "parsed" in obj:
                answers.append(obj["parsed"])
    return answers

In [3]:
# Structured answers
yaml_standard_answers = read_yaml_answers("llc_results/test_yaml.yaml")
yaml_artist_answers = read_yaml_answers("llc_results/test_yaml_artist.yaml")
yaml_chef_answers = read_yaml_answers("llc_results/test_yaml_chef.yaml")
yaml_detective_answers = read_yaml_answers("llc_results/test_yaml_detective.yaml")
yaml_judge_answers = read_yaml_answers("llc_results/test_yaml_judge.yaml")
# Unstructured answers
unstructured_standard_answers = read_unstructured_parsed("llc_results/test_unstructured_parsed.json")
unstructured_artist_answers = read_unstructured_parsed("llc_results/test_unstructured_artist_parsed.json")
unstructured_chef_answers = read_unstructured_parsed("llc_results/test_unstructured_chef_parsed.json")
unstructured_detective_answers = read_unstructured_parsed("llc_results/test_unstructured_detective_parsed.json")
unstructured_judge_answers = read_unstructured_parsed("llc_results/test_unstructured_judge_parsed.json")

In [4]:
with open("data/llc_test_revised.json", "r") as f:
    label = json.load(f)
unstructured_standard_results = []
unstructured_artist_results = []
unstructured_chef_results = []
unstructured_detective_results = []
unstructured_judge_results = []

yaml_standard_results = []
yaml_artist_results = []
yaml_chef_results = []
yaml_detective_results = []
yaml_judge_results = []

for i in range(len(unstructured_standard_answers)):
    label_tmp = label[i]['answer']
    # Unstructured answers
    if label_tmp.lower() == unstructured_standard_answers[i].lower():
        unstructured_standard_results.append(1)
    else:
        unstructured_standard_results.append(0)
        
    if label_tmp.lower() == unstructured_artist_answers[i].lower():
        unstructured_artist_results.append(1)
    else:
        unstructured_artist_results.append(0)
        
    if label_tmp.lower() == unstructured_chef_answers[i].lower():
        unstructured_chef_results.append(1)
    else:
        unstructured_chef_results.append(0)
        
    if label_tmp.lower() == unstructured_detective_answers[i].lower():
        unstructured_detective_results.append(1)
    else:
        unstructured_detective_results.append(0)
        
    if label_tmp.lower() == unstructured_judge_answers[i].lower():
        unstructured_judge_results.append(1)
    else:
        unstructured_judge_results.append(0)
    
    # Structured answers
    if label_tmp.lower() == yaml_standard_answers[i].lower():
        yaml_standard_results.append(1)
    else:
        yaml_standard_results.append(0)   
        
    if label_tmp.lower() == yaml_artist_answers[i].lower():
        yaml_artist_results.append(1)
    else:
        yaml_artist_results.append(0) 
        
    if label_tmp.lower() == yaml_chef_answers[i].lower():
        yaml_chef_results.append(1)
    else:
        yaml_chef_results.append(0) 
        
    if label_tmp.lower() == yaml_detective_answers[i].lower():
        yaml_detective_results.append(1)
    else:
        yaml_detective_results.append(0) 
        
    if label_tmp.lower() == yaml_judge_answers[i].lower():
        yaml_judge_results.append(1)
    else:
        yaml_judge_results.append(0) 

In [5]:
# Test whether output format significantly influences LLMs' generation
print(f"Averaged accuracy of unstructured format with diverse instruction: {format((sum(unstructured_standard_results)+sum(unstructured_artist_results)+sum(unstructured_chef_results)+sum(unstructured_detective_results)+sum(unstructured_judge_results))/(5*len(unstructured_standard_results)),'.3f')}")
print(f"Averaged accuracy of YAML format with diverse instruction: {format((sum(yaml_standard_results)+sum(yaml_artist_results)+sum(yaml_chef_results)+sum(yaml_detective_results)+sum(yaml_judge_results))/(5*len(yaml_standard_results)),'.3f')}")
data_subsets = [
    # Stratum of control on standard instruction
    (np.array(yaml_standard_results), np.array(unstructured_standard_results)),
    # Stratum of control on artist instruction
    (np.array(yaml_artist_results), np.array(unstructured_artist_results)),
    # Stratum of control on chef instruction
    (np.array(yaml_chef_results), np.array(unstructured_chef_results)),
    # Stratum of control on detective instruction
    (np.array(yaml_detective_results), np.array(unstructured_detective_results)),
    # Stratum of control on judge instruction
    (np.array(yaml_judge_results), np.array(unstructured_judge_results)),
]

p_values = []

for i, (correct_D, correct_E) in enumerate(data_subsets, 1):
    # Build contingency table for this stratum
    table = np.zeros((2, 2), dtype=int)
    for d, e in zip(correct_D, correct_E):
        table[d, e] += 1

    # Run McNemar's test
    result = mcnemar(table, exact=True)
    p_values.append(result.pvalue)

    # print(f"Stratum {i} contingency table:\n{table}")
    # print(f"Stratum {i} McNemar p-value: {result.pvalue:.5f}\n")

# Combine p-values using Stouffer’s method with correlation adjustment
# Convert p-values to Z-scores (two-sided)
z_scores = norm.isf(np.array(p_values) / 2)  # inverse survival function

# Assume weights (e.g., uniform or by number of samples per stratum)
weights = np.ones_like(z_scores)
# Assume correlation between strata (due to shared samples)
rho = 0.3
k = len(p_values)
R = np.full((k, k), rho)
np.fill_diagonal(R, 1)  # correlation matrix

# Stouffer's Z with correlation adjustment
numerator = np.sum(weights * z_scores)
denominator = np.sqrt(np.dot(weights, R @ weights))
z_combined = numerator / denominator
p_combined = 2 * norm.sf(abs(z_combined))

# print(f"Combined Stouffer Z: {z_combined:.3f}")
print(f"Combined p-value across strata (Stouffer’s method): {p_combined:.3f}")

Averaged accuracy of unstructured format with diverse instruction: 0.940
Averaged accuracy of YAML format with diverse instruction: 0.977
Combined p-value across strata (Stouffer’s method): 0.039


In [6]:
# Test whether instruction significantly influences LLMs' generation
print(f"Accuracy of unstructured & YAML format with standard instruction: {format((sum(unstructured_standard_results)+sum(yaml_standard_results))/(2*len(yaml_standard_results)),'.3f')}")
print(f"Averaged accuracy of unstructured & YAML format with intervened instruction: {format((sum(unstructured_artist_results)+sum(unstructured_chef_results)+sum(unstructured_detective_results)+sum(unstructured_judge_results)+sum(yaml_artist_results)+sum(yaml_chef_results)+sum(yaml_detective_results)+sum(yaml_judge_results))/(8*len(yaml_standard_results)),'.3f')}")

stratum_yaml = np.array([yaml_standard_results, yaml_artist_results, yaml_chef_results, yaml_detective_results, yaml_judge_results]).transpose()
stratum_unstructured = np.array([unstructured_standard_results, unstructured_artist_results, unstructured_chef_results, unstructured_detective_results, unstructured_judge_results]).transpose()
data_strata = [stratum_yaml, stratum_unstructured]

# Run Cochran's Q test on each stratum
p_values = []
# print("Cochran's Q test per stratum:")
for i, data in enumerate(data_strata):
    result = cochrans_q(data)
    p_values.append(result.pvalue)
    # print(f"  Stratum {i+1}: Q = {result.statistic:.4f}, p = {result.pvalue:.5f}")

# Combine p-values using Stouffer’s method with correlation adjustment
# Convert p-values to Z-scores (two-sided)
z_scores = norm.isf(np.array(p_values) / 2)  # inverse survival function

# Assume weights (e.g., uniform or by number of samples per stratum)
weights = np.ones_like(z_scores)
# Assume correlation between strata (due to shared samples)
rho = 0.3
k = len(p_values)
R = np.full((k, k), rho)
np.fill_diagonal(R, 1)  # correlation matrix

# Stouffer's Z with correlation adjustment
numerator = np.sum(weights * z_scores)
denominator = np.sqrt(np.dot(weights, R @ weights))
z_combined = numerator / denominator
p_combined = 2 * norm.sf(abs(z_combined))

# print(f"Combined Stouffer Z: {z_combined:.3f}")
print(f"Combined p-value across strata (Stouffer’s method): {p_combined:.3f}")

Accuracy of unstructured & YAML format with standard instruction: 0.953
Averaged accuracy of unstructured & YAML format with intervened instruction: 0.960
Combined p-value across strata (Stouffer’s method): 0.099


In [7]:
# If statistical significance threshold is 0.1
# Test conditional association between output format and instruction type
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM
# Transform yaml_standard_results into dataframe
# Comprising output format, instruction, and stratum of generation quality
# Output format: 1 structured 0 unstructured
# Instruction: 'a' standard 'b' artist 'c' chef 'd' detective 'e' judge
# Generation qiality: 1 true 0 false
df = pd.DataFrame({
    'Z': np.concatenate([np.repeat('a', len(yaml_standard_results)),
                         np.repeat('b', len(yaml_artist_results)),
                         np.repeat('c', len(yaml_chef_results)),
                         np.repeat('d', len(yaml_detective_results)),
                         np.repeat('e', len(yaml_judge_results)),
                         # Unstructured
                         np.repeat('a', len(unstructured_standard_results)),
                         np.repeat('b', len(unstructured_artist_results)),
                         np.repeat('c', len(unstructured_chef_results)),
                         np.repeat('d', len(unstructured_detective_results)),
                         np.repeat('e', len(unstructured_judge_results)),
                        ]),
    'X': np.concatenate([np.repeat(1, len(yaml_standard_results)),
                         np.repeat(1, len(yaml_artist_results)),
                         np.repeat(1, len(yaml_chef_results)),
                         np.repeat(1, len(yaml_detective_results)),
                         np.repeat(1, len(yaml_judge_results)),
                         # Unstructured
                         np.repeat(0, len(unstructured_standard_results)),
                         np.repeat(0, len(unstructured_artist_results)),
                         np.repeat(0, len(unstructured_chef_results)),
                         np.repeat(0, len(unstructured_detective_results)),
                         np.repeat(0, len(unstructured_judge_results)),
                        ]),
    'Y': np.concatenate([np.repeat(0, len(np.where(np.array(yaml_standard_results)==0)[0])),
                         np.repeat(1, len(np.where(np.array(yaml_standard_results)==1)[0])),
                         np.repeat(0, len(np.where(np.array(yaml_artist_results)==0)[0])),
                         np.repeat(1, len(np.where(np.array(yaml_artist_results)==1)[0])),
                         np.repeat(0, len(np.where(np.array(yaml_chef_results)==0)[0])),
                         np.repeat(1, len(np.where(np.array(yaml_chef_results)==1)[0])),
                         np.repeat(0, len(np.where(np.array(yaml_detective_results)==0)[0])),
                         np.repeat(1, len(np.where(np.array(yaml_detective_results)==1)[0])),
                         np.repeat(0, len(np.where(np.array(yaml_judge_results)==0)[0])),
                         np.repeat(1, len(np.where(np.array(yaml_judge_results)==1)[0])),
                         # Unstructured
                         np.repeat(0, len(np.where(np.array(unstructured_standard_results)==0)[0])),
                         np.repeat(1, len(np.where(np.array(unstructured_standard_results)==1)[0])),
                         np.repeat(0, len(np.where(np.array(unstructured_artist_results)==0)[0])),
                         np.repeat(1, len(np.where(np.array(unstructured_artist_results)==1)[0])),
                         np.repeat(0, len(np.where(np.array(unstructured_chef_results)==0)[0])),
                         np.repeat(1, len(np.where(np.array(unstructured_chef_results)==1)[0])),
                         np.repeat(0, len(np.where(np.array(unstructured_detective_results)==0)[0])),
                         np.repeat(1, len(np.where(np.array(unstructured_detective_results)==1)[0])),
                         np.repeat(0, len(np.where(np.array(unstructured_judge_results)==0)[0])),
                         np.repeat(1, len(np.where(np.array(unstructured_judge_results)==1)[0])),
                        ])
})

In [8]:
df_dummy = pd.get_dummies(df, columns=['Z'], drop_first=True)
# Define model components
endog = df_dummy['X']
exog = sm.add_constant(df_dummy[[col for col in df_dummy.columns if col.startswith('Z_')]])  # fixed effects
groups = df_dummy['Y']
exog_re = pd.get_dummies(groups)  # random intercepts per stratum
ident = np.ones(exog_re.shape[1], dtype=int)
# Fit mixed logistic model
model = BinomialBayesMixedGLM(endog, exog, exog_re, ident=ident)
result = model.fit_vb()
# Print results
print(result.summary())

             Binomial Mixed GLM Results
      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
----------------------------------------------------
const    M    -0.3930   0.0517                      
Z_b      M    -0.0124   0.1153                      
Z_c      M    -0.0012   0.1155                      
Z_d      M    -0.0152   0.1153                      
Z_e      M     0.0016   0.1155                      
VC_1     V     0.0000   1.0000 1.000   0.135   7.389
VC_2     V    -0.3836   0.4861 0.681   0.258   1.801
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


In [9]:
# Extract coefficient names, values, and std errors
param_names = model.exog_names  # names of fixed effects (same order as fe_mean)
coefs = result.fe_mean          # posterior means of fixed effects
ses = result.fe_sd              # posterior standard deviations

# Compute z-scores and p-values
z_scores = coefs / ses
p_values = 2 * (1 - norm.cdf(np.abs(z_scores)))  # two-tailed p-values

# Display results
for name, coef, se, z, p in zip(param_names, coefs, ses, z_scores, p_values):
    print(f"{name:20} coef = {coef: .4f}, SE = {se:.4f}, z = {z:.2f}, p = {p:.4f}")

const                coef = -0.3930, SE = 0.0517, z = -7.61, p = 0.0000
Z_b                  coef = -0.0124, SE = 0.1153, z = -0.11, p = 0.9141
Z_c                  coef = -0.0012, SE = 0.1155, z = -0.01, p = 0.9917
Z_d                  coef = -0.0152, SE = 0.1153, z = -0.13, p = 0.8948
Z_e                  coef =  0.0016, SE = 0.1155, z = 0.01, p = 0.9888
